# 0. Install and Import Dependencies

In [6]:
import os
import glob
from typing import List

import numpy as np
from matplotlib import pyplot as plt

import cv2
import sys

import tensorflow as tf

2024-02-03 10:48:05.542096: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-03 10:48:05.557712: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 10:48:05.557725: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 10:48:05.557736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-03 10:48:05.560975: I tensorflow/core/platform/cpu_feature_g

In [7]:
floder = ".."

# 1. Build Data Loading Functions

In [8]:
vocab = [x for x in "กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮๆะาิีึืใไเแ่้็๊๋โฤ์ฯุูำั123456789"] #define 

char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None, oov_token="")
num_to_char = tf.keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, oov_token="", invert=True)

2024-02-03 10:48:06.285403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-02-03 10:48:06.285430: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: lab-4090-B
2024-02-03 10:48:06.285436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: lab-4090-B
2024-02-03 10:48:06.285490: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.154.5
2024-02-03 10:48:06.285505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.129.3
2024-02-03 10:48:06.285508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.129.3 does not match DSO version 535.154.5 -- cannot find working devices in this configuration


In [9]:
def load_video(path: str):
    """convert video to list of frame
    path : path of video """
    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        frames.append(frame)

    cap.release()
    
    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames - mean), tf.float32) / std #nomalization

def load_alignments(path: str):
    """load data alignments in format text files alignment
    path : path of alignments"""
    
    path = path.split('/')[-1]
    path = path.split('-', 1)[-1]
    path = path[::-1].split('.', 1)[-1]
    file_name = path[::-1]
    return char_to_num(tf.reshape(tf.strings.unicode_split(file_name, input_encoding='UTF-8'), (-1)))
# def load_alignments(path: str):
#     """load data alignments in format text files alignment
#     path : path of alignments
#     file fomat  0 0 sil 
#                 0 20418 กระบวน 
#                 ...
#                 128727 352800 sil
#     """
#     with open(path, 'r', encoding='utf-8') as f:
#         lines = f.readlines()
#     tokens = []
#     for line in lines:
#         line = line.split() # covert string space to list
#         if line[2] != 'sil':
#             tokens = [*tokens, line[2]]
#     print(tokens)
#     return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))


def load_data(path: str):
    """load data to convert video and download file alignment"""
    path = bytes.decode(path.numpy())
    
    videonum = path.split('/')[-2]

    path = path.split('/')[-1]
    path = path[::-1].split('.', 1)[-1]
    file_name = path[::-1]

    


    video_path = f'{floder}/data/s1/{videonum}/{file_name}.mp4'
    alignment_path = f'{floder}/data/alignments/{videonum}/word-level/{file_name}.align'


    frames = load_video(video_path)
    alignments = load_alignments(alignment_path)

    if len(alignments) > 78:
        os.remove(video_path)
    return frames, alignments

In [10]:
def mappable_function(path:str) ->List[str]:
    """map data video and alignments file
    path : path of alignment or video"""
    result = tf.py_function(load_data, [path],  (tf.float32, tf.int64))
    
    if len(result[1]) > 78:
        tf.print(path)
    return result

# 2. Create Data Pipeline

In [11]:
data = tf.data.Dataset.list_files(f'{floder}/data/s1/*/*.mp4')
data = data.shuffle(500, reshuffle_each_iteration=False)
data = data.map(mappable_function)
print('original data : ', (tf.data.experimental.cardinality(data).numpy()))
for batch in data:
    try:
        tensor1, tensor2 = batch
        if  tensor2.shape[0] > 78:
            print(tensor2.shape[0])
    except cv2.error as e:
        print(f"OpenCV Error: {e}")
data = data.padded_batch(2, padded_shapes=([200,None, None,None],[79]), drop_remainder=True)
# print('padded data : ', (tf.data.experimental.cardinality(data).numpy()))
data = data.prefetch(tf.data.AUTOTUNE)
# Added for split train  & train
train = data.take(26000)
#test = data.skip(100)
test = data.skip(26000)
train_length = tf.data.experimental.cardinality(train).numpy()

# Get the length of the `test` dataset
test_length = tf.data.experimental.cardinality(test).numpy()

print("Length of train dataset:", train_length)
print("Length of test dataset:", test_length)

original data :  58211
Length of train dataset: 26000
Length of test dataset: 3105


In [5]:
import os
listdir = ["../data/s1/video-6351/247- ป่านนี้แล้ว เจ้ายังคิดเรื่องที่ต้องชดใช้ความผิดที่ทำให้ขาของข้าพิการอยู่อีกหรือ .mp4",
           "../data/s1/video-31323/50-คือเหมือนเดิมเหมือนกับกรณีของกูชาร์จเลย ก็คือต้องนำผู้ป่วยที่โดนต่อยเนี่ยออกมา  .mp4",
           "../data/s1/video-26219/32-มากขึ้นนะครับแม่เป็นแบบนี้นะก็ก็เราเป็น ไอเดียดูนะครับก็ไม่ได้บอกว่าของผมมันถูก  .mp4",
           "../data/s1/video-26219/3-อิสรภาพทางการเงินมากมันคือการมีรายได้ จากทรัพย์สินคือค่าเช่าดอกเบี้ยเงินปันผล  .mp4",
           "../data/s1/video-41540/249-คุณจะต้องรู้ว่าพลังงานของคุณ ไม่เลือกใช้น้ำตาลกับแป้ง ก็ใช้ของมัน ของทอด ถูกไหม.mp4",
           "../data/s1/video-41978/16-หุ้นแต่ละแบบมันมีข้อแตกต่างกันค่อนข้างเยอะนะครับอยู่ที่ว่าสไตล์เราจะเป็นสไตล์ไหน.mp4",
           "../data/s1/video-29701/33-ก็ถูกปิดตายทางเดินแล้วก็ทิ้งไว้ในท้องอยู่เฉยเฉยเลยนะคะตัวกระเพาะน้อยที่เหลืออยู่.mp4",
           "../data/s1/video-17327/156-เตรียมความพร้อมเรื่องด้านจิตใจมาละกันเพราะว่าเรามาอยู่กรุงเทพฯ เราตัวคนเดียวใช่ไหม.mp4",
           "../data/s1/video-41518/249-คุณจะต้องรู้ว่าพลังงานของคุณ ไม่เลือกใช้น้ำตาลกับแป้ง ก็ใช้ของมัน ของทอด ถูกไหม.mp4",
           "../data/s1/video-31323/49-คงชนะไม่ได้ใช้ช่วยแล้วนี้วิธีการต่อไปนะ ก็คือวิธีการดูแลผู้ป่วยเบื้องต้นนั้นก็  .mp4",
           "../data/s1/video-23317/10-การดูถูกของคนไซปรัสต่อคนเอเชียอย่างเรา แม่บ้านไซปรัสยังไม่ได้เห็นด้วยตัวเองนะ คะ  .mp4",
           "../data/s1/video-15213/273-แต่ถ้าสมมติว่าเราไปสังเกตภาษาอังกฤษที่วิวขึ้นให้ดูหรือว่าที่วิวพยายามตีความให้ดู.mp4",
           "../data/s1/video-41521/42-เพราะฉะนั้น ระหว่างทาง ถ้าคุณไม่ได้เอาน้ําตาลไปเก็บเป็นไขมัน คุณจะเป็นเบาหวานเลย.mp4",
           "../data/s1/video-40614/200-พระอาทิตย์กลัวจะมีปัญหานะคะ ก็เลยตัดสินใจว่าเออชุบชีวิตหนุมานขึ้นมาคุยกันดีกว่า.mp4",
           "../data/s1/video-41541/42-เพราะฉะนั้น ระหว่างทาง ถ้าคุณไม่ได้เอาน้ําตาลไปเก็บเป็นไขมัน คุณจะเป็นเบาหวานเลย.mp4",
           "../data/s1/video-266/26-ก็จะเห็นว่าเทคโนโลยีด้านเวลา ด้านอะไรเนี่ยก็จะเกี่ยวพันกับการเดินเรือเช่นกันค่ะ.mp4","../data/s1/video-20714/190-ห้าเล่มเนี้ยถือว่าเยอะมากมากนะครับ เพราะจริงจริงเล่มหนึ่งมันก็หลายตังค์อยู่นะฮะ.mp4"]
for i in listdir:
    try:
        os.remove(i)
    except:
        print("h")

h
h
h
h
h
h
h
h
h
h
h
h
h
h
h
h
h


In [33]:
train_length = tf.data.experimental.cardinality(train).numpy()

# Get the length of the `test` dataset
test_length = tf.data.experimental.cardinality(test).numpy()

print("Length of train dataset:", train_length)
print("Length of test dataset:", test_length)

Length of train dataset: 26000
Length of test dataset: 3114


In [34]:
test2 = data.as_numpy_iterator()

In [35]:
val = test2.next(); val[1]

array([[37, 68, 25, 25, 68, 57, 25, 25, 47,  7, 30, 48, 33,  1, 58, 54,
        36, 34, 61, 20, 25,  2, 65, 25, 10, 57, 47,  7,  2, 56, 33,  2,
        51, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [53, 43,  1, 47, 35, 54,  8, 47, 46,  0, 36,  7, 53, 27, 33, 68,
        25,  4, 51, 43, 43, 46, 53, 35,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

# 3. Design the Deep Neural Network

In [36]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import EarlyStopping

In [37]:
# designing model
model = Sequential()
#model.add(Conv3D(128, 3, input_shape=(75,,None,1), padding='same'))
model.add(Conv3D(128, 3, input_shape=(200, 46, 140, 1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(256, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(200, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(TimeDistributed(Flatten()))

model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Dense(char_to_num.vocabulary_size()+1, kernel_initializer='he_normal', activation='softmax'))

In [38]:
val[0].shape

(2, 200, 46, 140, 1)

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 200, 46, 140, 12   3584      
                             8)                                  
                                                                 
 activation (Activation)     (None, 200, 46, 140, 12   0         
                             8)                                  
                                                                 
 max_pooling3d (MaxPooling3  (None, 200, 23, 70, 128   0         
 D)                          )                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 200, 23, 70, 256   884992    
                             )                                   
                                                                 
 activation_1 (Activation)   (None, 200, 23, 70, 256   0

# 4. Setup Training Options and Train

In [40]:
class LossHistoryAndSave(keras.callbacks.Callback):
    def __init__(self, file_path):
        super().__init__()
        self.losses = []
        self.val_losses = []
        self.file_path = file_path

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
        # Save loss values to a file
        with open(self.file_path, 'a') as file:
            file.write(f'Epoch {epoch + 1}: Training Loss: {logs.get("loss")}, Validation Loss: {logs.get("val_loss")}\n')

class ProduceExample(tf.keras.callbacks.Callback): 
    def __init__(self, dataset) -> None: 
        self.dataset = dataset.as_numpy_iterator()
    
    def on_epoch_end(self, epoch, logs=None) -> None:
        data = self.dataset.next()
        yhat = self.model.predict(data[0])
        decoded = tf.keras.backend.ctc_decode(yhat, [200,200], greedy=False)[0][0].numpy()
        for x in range(len(yhat)):           
            print('Original:', tf.strings.reduce_join(num_to_char(data[1][x])).numpy().decode('utf-8'))
            print('Prediction:', tf.strings.reduce_join(num_to_char(decoded[x])).numpy().decode('utf-8'))
            print('~'*100)

def CTCLoss(y_true, y_pred):
    """define loss evaluation
    y_true : ground true
    y_pred : model predict data
    """
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")

    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")
    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    #ignore_longer_outputs_than_inputs=True
    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

def scheduler(epoch, lr):
    """learning rate
    epoch : epoch each dataset
    lr : learning rate
    """
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [41]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=CTCLoss)
checkpoint_callback = ModelCheckpoint(f'{floder}/checkpoint_1/checkpoint', monitor='val_loss',save_weights_only=True)
schedule_callback = LearningRateScheduler(scheduler)
loss_history_and_save = LossHistoryAndSave(f"{floder}/loss_log/loss_history.txt")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"{floder}/logs")
example_callback = ProduceExample(test)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.load_weights(f'../data/checkpoint_1/checkpoint')


In [42]:
model.fit(train, validation_data=test, epochs=100, batch_size=64, callbacks=[example_callback, early_stopping, tensorboard_callback, loss_history_and_save, checkpoint_callback, schedule_callback])

Epoch 1/100
    7/26000 [..............................] - ETA: 31:16:27 - loss: 104.9451

   69/26000 [..............................] - ETA: 31:45:59 - loss: 111.4725

# 5. Make a Prediction

In [ ]:
#url = 'https://drive.google.com/uc?id=1vWscXs4Vt0a_1IH1-ct2TCgXAZT-N3_Y'
#output = 'checkpoints.zip'
#gdown.download(url, output, quiet=False)
#gdown.extractall('checkpoints.zip', 'models')

In [ ]:
# from tensorflow.keras.optimizers import legacy

# # define your optimizer as a legacy optimizer
# optimizer = legacy.Adam()

# model.compile(optimizer=optimizer)

In [ ]:
# model.load_weights('/home/somkiat/Lip_reading/checkpoint_1/checkpoint')
model.load_weights(f'../data/checkpoint_1/checkpoint')

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
sample = test_data.next()

In [ ]:
eva_true = []
eva_pred = []
for sample in test_data:
    try:
        for i in range(0, 2):
            y_true = []
            for j in sample[1][i]:
                if j != 0:
                    y_true.append(j)
            yhat = model.predict(sample[0])
            eva_true.append(''.join([string.decode('utf-8') for string in num_to_char(y_true).numpy()]))                                                                                      
            y_pred = []
            for k in yhat[i]:
                sorted_array = np.sort(k)[::-1]
                # Get the fifth largest value
                top_five_indices = np.argsort(k)[-5:]

                # Replace values not in the top five with 0
                result_array = np.where(np.isin(np.arange(len(k)), top_five_indices), k, 0)
                top_five_indices = np.argsort(result_array)[-5:]
                result_array = np.zeros(np.max(top_five_indices) + 1)
                result_array[top_five_indices] = top_five_indices  # Y
                result_array = result_array[result_array != 0]
                top_five = []
                for x in result_array:
                    top_five.append(x)
                y_pred.append(top_five)
            top_one = []
            for m in range(0, len(y_true)):
                if y_true[m] in y_pred[m]:
                    top_one.append(y_true[m])
                else:
                    # top_one.append(max)
                    top_one.append(79)
            eva_pred.append(''.join([string.decode('utf-8') for string in num_to_char(top_one).numpy()]))
    except:
        pass

2023-11-25 10:01:11.677306: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-25 10:01:11.715210: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-25 10:01:11.770591: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-25 10:01:11.770614: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-25 10:01:11.770626: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate

1/1 [==============================] - 0s 54ms/step


In [ ]:
eva_pred

['แล้ว็นการัาน',
 'แล้วเมงไกน',
 'ใราน',
 'นแา',
 'ใ้น่อง',
 'น้องเป็นแ',
 'กรัะค',
 'ะ่นาค',
 'เาะวก็อคิ',
 'แต่รายะัััง่',
 'งมกา',
 'น',
 'งก',
 'ัอยาน',
 'แก้อนจั',
 'แต่วรก',
 'แั',
 'แา',
 'ี',
 'เกิัน',
 'ราร',
 'กัก',
 'เะกเอกกกนนกายริง',
 'วมา',
 'นทกันเย',
 'เาั่มัน',
 'ัใก้ม',
 'นูน',
 'ั่ัรเรื่อง',
 'วใัเมน',
 '่ารก',
 '',
 '่เกมนงเนี่',
 'าง',
 'งงเป็นน',
 'เล่ยแนั่',
 'ไม่ระ',
 'นะ',
 '็ก',
 '',
 'เาะ้กน่า',
 'เ่่อ่งไ',
 'ร',
 'ียนเ',
 'ยาไ้ย',
 'แตรวเร็',
 'เ็นที่',
 'น',
 'กอะไร',
 'ง',
 'น่ค',
 'แต่กนมั่ายบ',
 'แล้วกว',
 'นากเ',
 'เีวาน',
 'แต่วระแกกน',
 'แล้วเมานน',
 'เ้าเนะค',
 'ใร่',
 'แต่ว',
 'เ็นักาา',
 'าวย',
 'ันี้ะ',
 'ไมแที่แล',
 'เาะ้อะไราย',
 'นเ',
 'เงารืมีแ',
 'นะ',
 'แต็นก',
 'แลวเนี่มร',
 'ะ',
 'ยน',
 'เาะวั่มานแนนก',
 'าอย',
 'าา',
 'าเไมกน',
 'ั',
 'กเ',
 'เนงค',
 'แตกวะ',
 'ะูค',
 'แ่างเ',
 'แลวากกาน',
 'แล้วเ็มีอไิาง',
 'แนน่',
 'เ็นว',
 'น',
 'าินบ่อ',
 'ะมี',
 'กับอ',
 'ไัวะ',
 'ะน',
 'แล้วค',
 'เ่า',
 'ะ',
 '่ะก',
 'แลกยไมรร',
 '่ทกนก',
 'ะราอา',


In [ ]:
print(len(eva_true))
print(len(eva_pred))

314
314


In [ ]:
eva = []
for i in range(0, len(eva_true)):
    error = cer(eva_true[i], eva_pred[i])
    eva.append(error)
# Print the result
print(f"Average Char Error Rate: {sum(eva) / len(eva)}")

Average Char Error Rate: 0.828020037906062


In [ ]:
# y_true = [string.decode('utf-8') for string in num_to_char(y_true).numpy()]
# y_pred = [string.decode('utf-8') for string in num_to_char(top_one).numpy()]

In [ ]:
import pandas as pd

# Creating a DataFrame
df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})

# Display the DataFrame
print(df.head(29))

ValueError: All arrays must be of the same length

In [ ]:
# yhat = model.predict(sample[0])
# for i in yhat:
#     index = i[81]
#     sorted_array = np.sort(index)[::-1]
#     # Get the fifth largest value
#     top_five_indices = np.argsort(index)[-5:]

#     # Replace values not in the top five with 0
#     result_array = np.where(np.isin(np.arange(len(index)), top_five_indices), index, 0)
#     top_five_indices = np.argsort(result_array)[-5:]
#     print(top_five_indices)
#     result_array = np.zeros(max(top_five_indices) + 1)
#     result_array[top_five_indices] = top_five_indices  # Y
#     result_array = result_array[result_array != 0]
#     for i in result_array:
#         print(int(i))
    

In [ ]:
# result_array

In [ ]:
# decoded = tf.keras.backend.ctc_decode(result_array, input_length=[200,200], greedy=True).numpy()

In [ ]:
# yhat = model.predict(sample[0])
# print(np.argmax(yhat[0][0]))
# print('~'*100, 'REAL TEXT')
# tf.print([tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in sample[1]])
# decoded = tf.keras.backend.ctc_decode(yhat, input_length=[200,200], greedy=True)[0][0].numpy()
# print('~'*100, 'PREDICTIONS')
# tf.print([tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded])  
# print(decoded[0])

1/1 [==============================] - 0s 56ms/step
78
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ REAL TEXT
["ความเกิดจากนี้", "หรือบางทีดูชื่อเว็บไซต์อย่างเดียวไม่พอค่ะ"]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ PREDICTIONS
["ะ", ""]
[46  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1]


In [ ]:
# #sample = test_data.next()
# for sample in test_data:
#     yhat = model.predict(sample[0])
#     print('~'*100, 'REAL TEXT')
#     tf.print([tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in sample[1]])
#     decoded = tf.keras.backend.ctc_decode(yhat, input_length=[200,200], greedy=True)[0][0].numpy()
#     print('~'*100, 'PREDICTIONS')
#     tf.print([tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded])    

In [ ]:
# print('~'*100, 'REAL TEXT')
# tf.print([tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in sample[1]])

In [ ]:
# decoded = tf.keras.backend.ctc_decode(yhat, input_length=[75,75], greedy=True)[0][0].numpy()

In [ ]:
# print('~'*100, 'PREDICTIONS')
# tf.print([tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded])

# Test on a Video

In [ ]:
#sample = load_data(tf.convert_to_tensor('.\\data\\s1\\bras9a.mpg'))

In [ ]:
# print('~'*100, 'REAL TEXT')
# [tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in [sample[1]]]

In [ ]:
# yhat = model.predict(tf.expand_dims(sample[0], axis=0))

In [ ]:
# decoded = tf.keras.backend.ctc_decode(yhat, input_length=[75], greedy=True)[0][0].numpy()

In [ ]:
# print('~'*100, 'PREDICTIONS')
# [tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded]